In [1]:
import numpy as np
from pygmo import fast_non_dominated_sorting as nds
from timeit import timeit

In [29]:
num_var = 7
num_samples=30
mean = 0.5
std_dev = 0.1
means = [mean] * num_var
cov = np.eye(num_var) * np.square(std_dev)
data = np.random.multivariate_normal(means, cov, num_samples)
data = np.vstack((data,data))

In [32]:
data = np.asarray([[1,1],[1,2],[2,2],[3,3],[2,4]])

In [35]:
nds(data)[0]

[array([0], dtype=uint64),
 array([1], dtype=uint64),
 array([2], dtype=uint64),
 array([3, 4], dtype=uint64)]

In [37]:
comp3 = (data > data[:, None]).any(axis=2)
comp4 = (data >= data[:,None]).all(axis=2)

i_dom_by_j = comp3 & comp4

taken = np.where(i_dom_by_j.sum(axis=0)==0)[0]
ranks = []
ranks_new = taken
left = data.shape[0]


while left:
    ranks.append(ranks_new)
    left = left - len(ranks_new)
    i_dom_by_j[ranks_new] = 0
    ranks_new = np.where(i_dom_by_j.sum(axis=0)==0)[0]
    ranks_new = np.setdiff1d(ranks_new, taken)
    taken = np.append(taken, ranks_new)
ranks

[array([0]), array([1]), array([2]), array([3, 4])]

In [2]:
setup = """
import numpy as np
from pygmo import fast_non_dominated_sorting as nds

num_var = 5
num_samples=2000
mean = 0.5
std_dev = 0.1
means = [mean] * num_var
cov = np.eye(num_var) * np.square(std_dev)
data = np.random.multivariate_normal(means, cov, num_samples)
"""

In [11]:
code1 = """nds(data)[0]"""


code2 = """
comp3 = data > data[:, None]
comp4 = data >= data[:,None]
i_dom_by_j = np.logical_and(np.any(comp3, axis=2), np.all(comp4, axis=2))
"""

code3 = """
comp3 = (data > data[:, None]).any(axis=2)
comp4 = (data >= data[:,None]).all(axis=2)
i_dom_by_j = comp3&comp4
"""

code4 = """
comp3 = (data > data[:, None]).any(axis=2)
comp4 = (data >= data[:,None]).all(axis=2)

i_dom_by_j = comp3 & comp4

taken = np.where(i_dom_by_j.sum(axis=0)==0)[0]
ranks = []
ranks_new = taken
left = data.shape[0]


while left:
    ranks.append(ranks_new)
    left = left - len(ranks_new)
    i_dom_by_j[ranks_new] = 0
    ranks_new = np.where(i_dom_by_j.sum(axis=0)==0)[0]
    ranks_new = np.setdiff1d(ranks_new, taken)
    taken = np.append(taken, ranks_new)
    """
code5 = """comp3 = data > data[:, None]
comp4 = data >= data[:,None]
i_dom_by_j = (comp3 & comp4).all(axis=2)"""

code6 = """
comp3 = data > data[:, None]
comp4 = data >= data[:,None]
i_dom_by_j = (comp3 & comp4).all(axis=2)

taken = np.where(i_dom_by_j.sum(axis=0)==0)[0]
ranks = []
ranks_new = taken
left = data.shape[0]


while left:
    ranks.append(ranks_new)
    left = left - len(ranks_new)
    i_dom_by_j[ranks_new] = 0
    ranks_new = np.where(i_dom_by_j.sum(axis=0)==0)[0]
    ranks_new = np.setdiff1d(ranks_new, taken)
    taken = np.append(taken, ranks_new)
"""

In [73]:
timeit(code1, setup, number=10)

0.9377404320002825

In [14]:
timeit(code2, setup, number=10)

1.842095836000226

In [15]:
timeit(code3, setup, number=10)

1.8395706219998829

In [74]:
timeit(code4, setup, number=10)

2.0862643840009696

In [16]:
timeit(code5, setup, number=10)

1.2949060640021344

In [18]:
timeit(code6, setup, number=10)

1.6276612289984769

In [221]:
timeit("data>data[:,None]", setup, number=10)

0.35281351400044514

In [222]:
timeit("(data>data[:,None]).any(axis=2)", setup, number=10)

0.920774430000165

In [219]:
timeit("np.logical_or.reduce((data>data[:,None]),axis=2)", setup, number=20)

1.8461779300014314

In [33]:
i_dom_by_j = np.empty((data.shape[0],data.shape[0]), dtype=bool)

In [42]:
for i, dat in enumerate(data):
    i_dom_by_j[i] = (dat > data).any(axis=1) & (dat >= data).all(axis=1)
test = i_dom_by_j * 1

In [37]:
comp3 = data > data[:, None]
comp4 = data >= data[:,None]

i_dom_by_j = np.logical_and(np.any(comp3, axis=2), np.all(comp4, axis=2)) * 1
test2 = i_dom_by_j

In [228]:
comp3 = (data > data[:, None]).any(axis=2)
comp4 = (data >= data[:,None]).all(axis=2)
a = comp3&comp4

In [232]:
comp3 = data > data[:, None]
comp4 = data >= data[:,None]
b = (comp3 & comp4).all(axis=2)

In [231]:
np.where(a==True)

(array([ 0,  0,  2,  2,  3,  3,  6,  6,  9, 10, 13, 13, 18, 18, 18, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26]),
 array([11, 15, 11, 21,  5, 21, 11, 22,  5,  5,  5, 17,  5, 17, 31,  1,  2,
         5,  7, 11, 16, 19, 20, 21, 22, 25, 28, 29, 30]))

In [233]:
np.where(b==True)

(array([ 0,  0,  2,  2,  3,  3,  6,  6,  9, 10, 13, 13, 18, 18, 18, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26]),
 array([11, 15, 11, 21,  5, 21, 11, 22,  5,  5,  5, 17,  5, 17, 31,  1,  2,
         5,  7, 11, 16, 19, 20, 21, 22, 25, 28, 29, 30]))

In [237]:
np.where((a!=b))

(array([], dtype=int64), array([], dtype=int64))

In [71]:
timeit("a.append(b)", "import numpy as np;a=[];b=np.asarray([1,3,4,5,6]*3000)")

0.03790868199939723

In [72]:
timeit("a+=b.shape[0]", "import numpy as np;a=0;b=np.asarray([1,3,4,5,6]*3000)")

0.09325137600171729